In [1]:
import logging

import folium
import geopandas
import shapely

from aipbuilder.build_airspaces import (
    parse_geometry_definition_file,
    parse_vfr_reporting_points,
)
from aipbuilder.providers import ProviderToken

log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


class ExampleBorderProvider:
    def __init__(self, path: str) -> None:
        self._gdf = geopandas.GeoDataFrame.from_file(path)
        pass

    def get_border(self, border_name: str) -> shapely.LineString:
        df = self._gdf.loc[self._gdf.name == border_name]
        if not len(df):
            raise KeyError(border_name)
        return df.iloc[0].geometry


providers = {ProviderToken.BORDER_PROVIDER: ExampleBorderProvider('./example-data/borders/example_borders.shp')}

In [2]:

gdf_airspaces = parse_geometry_definition_file(
    "./example-data/airspaces.csv", providers
)
gdf_restricted_areas = parse_geometry_definition_file(
    "./example-data/restricted_areas.csv", providers
)
gdf_danger_areas = parse_geometry_definition_file(
    "./example-data/danger_areas.csv", providers
)
gdf_fir = parse_geometry_definition_file("./example-data/fir.csv", providers)
gdf_vfr_reporting_points = parse_vfr_reporting_points(
    "./example-data/vfr_reporting_points.csv"
)

In [ ]:
m = folium.Map()
gdf_airspaces.explore(m=m, color="blue")
gdf_restricted_areas.explore(m=m, color="orange")
gdf_danger_areas.explore(m=m, color="yellow")
gdf_fir.explore(m=m, color="purple", style_kwds={"fill": False})
gdf_vfr_reporting_points.explore(
    m=m,
    marker_type="circle",
    marker_kwds={"color": "black", "fill": True, "radius": 250},
)
for border_name in ["CH", "LI"]:
    border_points = providers[ProviderToken.BORDER_PROVIDER].get_border(border_name).coords
    for border_point in border_points:
        folium.CircleMarker(
            (border_point[1], border_point[0]),
            radius=5,
            color="magenta",
            stroke=False,
            fill=True,
            fill_opacity=0.6,
            opacity=1,
        ).add_to(m)